# Summary of what is available in Data:

## With Numbers of Visit

### Numbers of visit in absolute scale
- <i>All_Nodes_During_WC.csv</i>: a table with every nodes, their categories and the number of visit per day during the World cup itself (the world cup started on 14 June and lasted until 15 July 2018). This starts on the 10th of June and ends on 20th of July (border included).
[<b>Remark: for <i>All_Nodes_During_WC.csv</i>, 2 problems occur at the nodes 209 and 572 (Peter Etebo and Ricardo Pereira). Their numbers of visits are set to 0.]


- <i>Countries_Enlarged_During_Year.csv</i>: [<b>for question 2</b>] any countries in any world cup in history. Number of visit per month during the year of the world cup (actually: from 14 November 2017 to (included) 14 November 2018. <b>IMPORTANT REMARK</b>: to help observe the world cup, start of month is set to 14 (since the world cup started the 14th June). The column indicates the month the time started. This is done because otherwise the world cup is cut in two parts. 


- <i>Countries_Enlarged_During_Year_centred.csv</i>: same as right above except it starts at the first of the month and goes from Decembre 2017 to Novembre 2018

### Numbers of visit in Normalised scale 
Same files as above but the names end with "\_Normalised" 


- <i>All_Nodes_During_WC_Normalised.csv</i>: as above but normalised. <b>Last column 'Sum' (index 43) is the total value !!</b>


- <i>Countries_Enlarged_During_Year_Normalised.csv</i>: as above but normalised. <b>Last column 'Sum' (index 14)is the total value !!</b>

## Without Number of Visits

- <i>All_Nodes.csv</i>: each nodes and its category <b>[useful to create a new csv but keep this one clean!]</b>


- <i>Nodes_Linked.csv</i>: each node enters the table with its links in <i>Links</i> and the category of the node in <i>Category</i>(the original one or, equivalently, that in the <i>Nodes</i> column). <b>Use this for the connected network base on hyperlinks</b> (do check that what you obtain is connected!)


## Supplement: 
These <b>should not be necessary</b> as they only contain the node and its category in seperated files for each category

Separeted Info:
- players in the 2018 World Cup in <i>All_players.csv</i>
- countries in the 2018 World Cup in <i>All_countries.csv</i>
- teams in the 2018 World Cup in All_<i>National_Teams.csv</i>

And in extra, with more entries than just that World Cup: [modified for question 2 in later]
- any countries in any world cup in <i>Countries_Enlarged.csv</i>


In [ ]:
import datetime
import requests
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
from collections import OrderedDict
import wikipedia

In [ ]:
#Only save if you're sure the file needs to be modified ! Please contact me first.
SAVE = 0
SAVE_WITH_LINKS = 0
SAVE_WITH_WC_VISITS = 0
SAVE_WITH_WC_VISITS_NORMALISED = 0
SAVE_COUNTRY_YEAR = 0
SAVE_COUNTRY_YEAR_NORMALISED = 0

# Pages: 

Players: https://en.wikipedia.org/w/index.php?title=Category:2018_FIFA_World_Cup_players&pagefrom=Layun%2C+Miguel%0AMiguel+Layún#mw-pages

Countries: https://en.wikipedia.org/wiki/Category:Countries_at_the_2018_FIFA_World_Cup

Teams? https://en.wikipedia.org/wiki/2018_FIFA_World_Cup_squads

In [ ]:
#To get the links from the category page
import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    'action': "query",
    'list': "categorymembers",
    'cmtitle': "Category:2018 FIFA World Cup players",
    'cmlimit': 800,
    'format': "json"
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

player_list = []
for i in range(len(DATA['query']['categorymembers'])):
    player_list.append(DATA['query']['categorymembers'][i]['title'])
"""
for player in player_list:
    print(player)
"""


In [ ]:
#Upload the last players (unfortunately method above is limited at 500)
last_players_pd = pd.read_csv("./Data/Process/Players_End.csv")

# Combine Both in a new list.
last_players = last_players_pd['Players'].values.tolist()
player_list.extend(last_players)
players = OrderedDict([ ('Players', player_list)])
player_pd = pd.DataFrame.from_dict(players)

if(SAVE):
    player_pd.to_csv('./Data/Process/All_Player_Dirty.csv', sep='\t', encoding= 'utf-16', index=False)

In [ ]:
#Based on the All_Player_Dirty and cleaned by Google Spreadsheet. Load this
#       player_pd  = pd.read_csv("./Data/All_Players.csv")


We now have all the data (we get the countries and the teams manually):
- players in All_players.csv
- countries in All_countries.csv
- teams in All_National_Teams.csv

And in extra, with more entries than just that world cup:
- any countries in any world cup in Countries_Enlarged.csv

We now have to find the links on these pages and the number of visit for each. Let's make a register where we store every nodes. 

REMARK: Countries_Enlarged.csv is not used since the links will only be based on visit and not hyperlinks.

In [ ]:
player_pd  = pd.read_csv("./Data/All_Players.csv")
countries_pd  = pd.read_csv("./Data/All_Countries.csv")
teams_pd  = pd.read_csv("./Data/All_National_Teams.csv")


In [ ]:
frames = [player_pd, countries_pd, teams_pd]
Nodes_Join = pd.concat(frames)
Nodes_Join.to_csv('./Data/All_Nodes.csv', sep='\t', encoding= 'utf-16', index=False)

For each of these nodes, need to run command to get all pages linked, and keep only those that match one of the node.Add these into a new dataframe.

In [ ]:
def print_links(page):
        links = page.links
        return links     

In [ ]:
def proccess_data_link(Nodes_Join):
    entry_nodes = Nodes_Join.Node.tolist()
    entry_category = Nodes_Join.Category.tolist()
    size = len(entry_nodes)
    final_nodes = []
    final_link = []
    final_category = []
    
    for i in range(size):
        if(i%50 == 0):
            print("At node", i)
        page_run = wikipedia.page(entry_nodes[i])
        links_from_page = print_links(page_test)
        links_from_page_in_nodes =  list(set(entry_nodes).intersection(links_from_page))
        for elem in links_from_page_in_nodes:
            final_nodes.append(entry_nodes[i])
            final_link.append(elem)
            final_category.append(entry_category[i])
    
    linked = OrderedDict([('Nodes', final_nodes),
          ('Links', final_link),
          ('Node_Category',  final_category)])
    
    df_linked = pd.DataFrame.from_dict(linked)
    
    return df_linked

In [ ]:
df =  proccess_data_link(Nodes_Join)

In [ ]:
if(SAVE_WITH_LINKS):
    df.to_csv('./Data/Nodes_Linked.csv', sep='\t', encoding= 'utf-16', index=False)

# Getting the Number of Visits

### Part 1: During the entire World Cup for every nodes

In [ ]:
def getUserActivity(article, granularity, start, end, project ="en.wikipedia.org",
                    access="all-access", agent="user",dateformat="iso"):
    """
    Method to obtain user activity of a given page for a given period of time
    article: name of the wikiipedia article
    granularity: time granularity of activity, either 'monthly' or 'daily'
    start: start date of the research as Datetime.datetime object
    end: end date of the research as Datetime.datetime object
    project: If you want to filter by project, use the domain of any Wikimedia project (by default en.wikipedia.org)
    access: If you want to filter by access method, use one of desktop, mobile-app or mobile-web (by default all-access)
    agent: If you want to filter by agent type, use one of user, bot or spider (by default user).
    dateformat: the dateformat used in result array, can be 'iso','ordinal','datetime'.
    return:
        it return an array of array of the form [ [user_activity_value1, date1], [user_activity_value2, date2]]
    """

    #granularity['monthly','daily']
    #format['iso','ordinal','datetime']
    #Be carefull, for daily granularity left bound date is included, for monthly granularity left bound date is excluded
    
    dstart = start.strftime("%Y%m%d")
    dend = end.strftime("%Y%m%d")
    path = ("https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/"+project
            +"/"+access+"/"+agent+"/"+article+"/"+granularity+"/"+dstart+"/"+dend)
    r = requests.get(path)
    res = []
    for i in range(len(r.json()['items'])):
        time_label = None
        if granularity == 'daily':
            time_label = (start + datetime.timedelta(days=i))
        else:
            time_label = (start + relativedelta(months=+i))
        if dateformat == 'iso':
            time_label = time_label.isoformat()
        elif dateformat == 'ordinal':
            time_label = time_label.toordinal()
            
        res.append([r.json()['items'][i]['views'],time_label])
    return res

In [ ]:
#Set of all nodes and their category. We will modify this database 
# several times to get what we want
Nodes_Join = pd.read_csv("./Data/All_Nodes.csv", sep='\t', encoding= 'utf-16')

In [ ]:
#First query: visit during the World Cup (slightly extended) from  10th June till 20th July.
s = datetime.datetime(year=2018,month=6,day=10)
e = datetime.datetime(year=2018,month=7,day=20)

In [ ]:
def perform_query(Nodes_Join, start_time, end_time, timestep):
    df = Nodes_Join.copy(deep=True)
    entry_nodes = df.Node.tolist()
    
    #Number of Days in between:
    #number_days = (end_time-start_time).days
    
    #We do case 0 here, then the rest in loop
    node_0 = entry_nodes[0]
    if(timestep == "daily"):
        time_list = getUserActivity(article=node_0,granularity="daily",\
                                start=start_time,end=end_time,dateformat="datetime")
    if(timestep == "monthly"):
        time_list = getUserActivity(article=node_0,granularity="monthly",\
                                start=start_time,end=end_time,dateformat="datetime")
    counter = 0
    for elem in time_list:
        if(timestep == "daily"):
            df[elem[1].strftime('%d/%m/%Y')] = ''
        if(timestep == "monthly"):
            df[elem[1].strftime('%m/%Y')] = ''
        df.iloc[0, 2 + counter] = elem[0]
        counter = counter +1
    #Case 0 done
    err_list = []
    #Start Loop
    for i in range(1, len(entry_nodes)):
        if(i % 50 == 0):
            print("At Node ", i)
        node_i = entry_nodes[i]
        try:
            if(timestep == "daily"):
                time_list = getUserActivity(article=node_i,granularity="daily",\
                                        start=start_time,end=end_time,dateformat="datetime")
            if(timestep == "monthly"):
                time_list = getUserActivity(article=node_i,granularity="monthly",\
                                        start=start_time,end=end_time,dateformat="datetime")
            counter = 0
            for elem in time_list:
                df.iloc[i, 2 + counter] = elem[0]
                counter = counter +1
        except:
            counter = 0
            for elem in time_list:
                df.iloc[i, 2 + counter] = 0
                counter = counter +1
            err_list.append(i)
            
        
    return df, err_list

In [ ]:
Nodes_During_WC, err_list = perform_query(Nodes_Join, s, e, "daily")

Problem occurs at the nodes 209 and 572 (Peter Etebo and Ricardo Pereira). The values is set to 0.

In [ ]:
if(SAVE_WITH_WC_VISITS):
    Nodes_During_WC.to_csv('./Data/All_Nodes_During_WC.csv', sep='\t', encoding= 'utf-16', index=False)

In [ ]:
All_Nodes_During_WC = pd.read_csv("./Data/All_Nodes_During_WC.csv", sep='\t', encoding= 'utf-16')


In [ ]:
#For Example
All_Nodes_During_WC[All_Nodes_During_WC['Node'] == "Lionel Messi"]

### Part 2: During the entire year (per month) for every country

In [ ]:
Contries_Enlarged  = pd.read_csv("./Data/Countries_Enlarged.csv")


In [ ]:
#Note how the day is set to 14th to help visualise the world cup and concentrate it in a month
s = datetime.datetime(year=2017,month=11,day=14)
e = datetime.datetime(year=2018,month=12,day=14)

In [ ]:
Countries_During_Year, err_list_Year = perform_query(Contries_Enlarged, s, e, "monthly")

No errors here (can check <i>err_list_Year</i>)

In [ ]:
if(SAVE_COUNTRY_YEAR):
    Countries_During_Year.to_csv('./Data/Countries_Enlarged_During_Year.csv', sep='\t', encoding= 'utf-16', index=False)
    

In [ ]:
#Contries_Enlarged_2  = pd.read_csv("./Data/Countries_Enlarged_During_Year.csv", sep='\t', encoding= 'utf-16')


### Part 3: Let's normalised these signals

In [ ]:
All_Nodes_During_WC = pd.read_csv("./Data/All_Nodes_During_WC.csv", sep='\t', encoding= 'utf-16')
All_Nodes_During_WC['Sum'] = All_Nodes_During_WC.sum(axis=1)

In [ ]:
for i in range (2, 43):
    All_Nodes_During_WC.iloc[:, i]= All_Nodes_During_WC.iloc[:, i]/All_Nodes_During_WC.iloc[:, 43]

In [ ]:
if(SAVE_WITH_WC_VISITS_NORMALISED):
    All_Nodes_During_WC.to_csv("./Data/All_Nodes_During_WC_Normalised.csv", sep='\t', encoding= 'utf-16', index=False)



In [ ]:
Countries_During_Year = pd.read_csv("./Data/Countries_Enlarged_During_Year.csv", sep='\t', encoding= 'utf-16')
Countries_During_Year['Sum'] = Countries_During_Year.sum(axis=1)


In [ ]:
for i in range (2, 14):
    Countries_During_Year.iloc[:, i]= Countries_During_Year.iloc[:, i]/Countries_During_Year.iloc[:, 14]
    

In [ ]:
if(SAVE_COUNTRY_YEAR_NORMALISED):
    Countries_During_Year.to_csv("./Data/Countries_Enlarged_During_Year_Normalised.csv", sep='\t', encoding= 'utf-16', index=False)



In [ ]:
#Countries_During_Year_2  = pd.read_csv("./Data/Countries_Enlarged_During_Year_Normalised.csv", sep='\t', encoding= 'utf-16')



<b>THE END</b> (and the start for you)